<a href="https://colab.research.google.com/github/johnnycleiton07/llm-studies/blob/main/Pre_processamento_com_NLTK_e_spaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pré-processamento de dados de texto com spaCy e NLTK

O pré-processamento de dados de texto com spaCy e NLTK combina as forças de ambas as bibliotecas para limpar e preparar os dados. O spaCy é eficiente na tokenização, lematização e reconhecimento de entidades, enquanto o NLTK oferece recursos adicionais como remoção de stop words e análise gramatical. Juntas, essas ferramentas permitem uma preparação de texto robusta, essencial para tarefas de NLP.

In [18]:
#nltk é nativa no colab, já spacy é necessário instalar
!pip install spacy
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 73.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [19]:
#importando as ferramentas necessárias
import pandas as pd
import numpy as np
import re
import nltk
import spacy
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load('pt_core_news_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


* Tokenização

In [24]:
data = {'text': ['Nós adoramos Python!', 'Python é legal!', 'Ciência de dados é legal', None, 'Estou correndo para aprender Python.']}
df = pd.DataFrame(data)
df

,text
0,Nós adoramos Python!
1,Python é legal!
2,Ciência de dados é legal
3,None
4,Estou correndo para aprender Python.


* Trocando valor de tokens "None" por valor vazio

In [25]:
df['text'].fillna('', inplace=True)
df

,text
0,Nós adoramos Python!
1,Python é legal!
2,Ciência de dados é legal
3,
4,Estou correndo para aprender Python.


*Remoção de números e pontuações

In [26]:
#limpeza de dados
def remove_noise(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

df['cleaned_text'] = df['text'].apply(remove_noise)
df

,text,cleaned_text
0,Nós adoramos Python!,nós adoramos python
1,Python é legal!,python é legal
2,Ciência de dados é legal,ciência de dados é legal
3,,
4,Estou correndo para aprender Python.,estou correndo para aprender python


* Tokenizando as células

In [27]:
df['tokenized_text'] = df['cleaned_text'].apply(nltk.word_tokenize)
df

,text,cleaned_text,tokenized_text
0,Nós adoramos Python!,nós adoramos python,"[nós, adoramos, python]"
1,Python é legal!,python é legal,"[python, é, legal]"
2,Ciência de dados é legal,ciência de dados é legal,"[ciência, de, dados, é, legal]"
3,,,[]
4,Estou correndo para aprender Python.,estou correndo para aprender python,"[estou, correndo, para, aprender, python]"


* Tratamento de stop words

In [28]:
stop_words = set(stopwords.words('portuguese'))
df['no_stopwords_text'] = df['tokenized_text'].apply(lambda x: [word for word in x if word not in stop_words])
df

,text,cleaned_text,tokenized_text,no_stopwords_text
0,Nós adoramos Python!,nós adoramos python,"[nós, adoramos, python]","[adoramos, python]"
1,Python é legal!,python é legal,"[python, é, legal]","[python, legal]"
2,Ciência de dados é legal,ciência de dados é legal,"[ciência, de, dados, é, legal]","[ciência, dados, legal]"
3,,,[],[]
4,Estou correndo para aprender Python.,estou correndo para aprender python,"[estou, correndo, para, aprender, python]","[correndo, aprender, python]"


In [29]:
print(stop_words)

{'houvéramos', 'o', 'entre', 'houveria', 'haver', 'se', 'para', 'lhe', 'sejam', 'tivéssemos', 'esses', 'eles', 'as', 'houvéssemos', 'das', 'temos', 'houvemos', 'tive', 'e', 'aquilo', 'hajam', 'aquelas', 'meus', 'depois', 'dele', 'fôramos', 'na', 'quando', 'nossas', 'nossos', 'delas', 'ele', 'esteja', 'estão', 'num', 'pela', 'muito', 'sem', 'aqueles', 'tiver', 'teríamos', 'houveríamos', 'tinha', 'tivemos', 'mesmo', 'já', 'ser', 'pelo', 'fôssemos', 'seu', 'com', 'seremos', 'a', 'tenhamos', 'elas', 'houverá', 'haja', 'nós', 'seria', 'pelos', 'era', 'seriam', 'eram', 'teu', 'você', 'lhes', 'isso', 'foram', 'estávamos', 'estivéramos', 'até', 'estamos', 'está', 'minhas', 'aquele', 'tua', 'tém', 'sejamos', 'minha', 'aos', 'serão', 'fora', 'seríamos', 'sua', 'estar', 'ou', 'de', 'teremos', 'do', 'houver', 'estou', 'estas', 'às', 'estivessem', 'houve', 'essas', 'vocês', 'um', 'houvesse', 'formos', 'tenham', 'nas', 'tivermos', 'terei', 'teriam', 'numa', 'houverei', 'estivemos', 'tínhamos', 'houv

* Steamming e Lematização

In [30]:
stemmer = PorterStemmer()
df['stemmed_text'] = df['no_stopwords_text'].apply(lambda x: [stemmer.stem(word) for word in x])

def lemmatize_text(text):
    doc = nlp(" ".join(text))
    return [token.lemma_ for token in doc]

df['lemmatized_text'] = df['no_stopwords_text'].apply(lemmatize_text)
df

,text,cleaned_text,tokenized_text,no_stopwords_text,stemmed_text,lemmatized_text
0,Nós adoramos Python!,nós adoramos python,"[nós, adoramos, python]","[adoramos, python]","[adoramo, python]","[adorar, python]"
1,Python é legal!,python é legal,"[python, é, legal]","[python, legal]","[python, legal]","[python, legal]"
2,Ciência de dados é legal,ciência de dados é legal,"[ciência, de, dados, é, legal]","[ciência, dados, legal]","[ciência, dado, legal]","[ciência, dado, legal]"
3,,,[],[],[],[]
4,Estou correndo para aprender Python.,estou correndo para aprender python,"[estou, correndo, para, aprender, python]","[correndo, aprender, python]","[correndo, aprend, python]","[correr, aprender, python]"


* vetorização do texto (one-hot encoding)

In [31]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['lemmatized_text'])
sequences = tokenizer.texts_to_sequences(df['lemmatized_text'])
one_hot_results = tokenizer.texts_to_matrix(df['lemmatized_text'], mode='binary')
one_hot_results

array([[0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 1.]])

In [32]:
word_index = tokenizer.word_index
print("Word Index:", word_index)

Word Index: {'python': 1, 'legal': 2, 'adorar': 3, 'ciência': 4, 'dado': 5, 'correr': 6, 'aprender': 7}


* word embedding (onde cada palavra é um vetor)

In [33]:
max_len = 10
padded_sequences = pad_sequences(sequences, maxlen=max_len)
padded_sequences

array([[0, 0, 0, 0, 0, 0, 0, 0, 3, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
       [0, 0, 0, 0, 0, 0, 0, 4, 5, 2],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 6, 7, 1]], dtype=int32)

* Contagem de palavras mais comuns

In [34]:
word_freq = Counter([word for sublist in df['lemmatized_text'] for word in sublist])
print("Palavras mais comuns:", word_freq.most_common(5))

Palavras mais comuns: [('python', 3), ('legal', 2), ('adorar', 1), ('ciência', 1), ('dado', 1)]


* Divisão de um conjunto de dados em subconjuntos de treinamento, validação e teste

In [35]:
X_train, X_temp, y_train, y_temp = train_test_split(df['lemmatized_text'], df['text'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print("Tamanho do conjunto de traino:", len(X_train))
print("Tamanho do cojunto de validação:", len(X_val))
print("Tamanho do conjunto de teste:", len(X_test))

Tamanho do conjunto de traino: 3
Tamanho do cojunto de validação: 1
Tamanho do conjunto de teste: 1
